[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/redcican/Coursera_Capstone/blob/master/Third_Assignment_Neighborhood_Segmentation_and_Clustering.ipynb)

# IBM Coursera Final Project 1: Segmenting and Clustering Neighborhoods in Toronto - Third Assignment

Explore and cluster the neighborhoods in Toronto.

First, import the csv file.

In [32]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

## 1. Import the Libraries

In [33]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
!pip install folium==0.5.0
import folium

print('Libraries imported!')

Libraries imported!


In [34]:
df = pd.read_csv('Toronto.csv')
df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


### Use geopy library to get the latitude and longitude values of Toronto.

In [35]:
!pip install geopy

In [0]:
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize

In [37]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f"The geographical coordinate of Toronto are {latitude}, {longitude}")

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geographical coordinate of Toronto are 43.653963, -79.387207


### Create a map of Toronto with neighborhoods superimposed on top.

In [38]:
# create map of Toronto using latitude and longitude values.
map_toronto = folium.Map(location=[latitude,longitude],zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'],df['Longitude'],df['Borough'],df['Neighbourhood']):
    label = f'{neighbourhood},{borough}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'fuchsia',
        fill = True,
        fill_colors = '#3186cc',
        fill_opacity = 0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

For illustration purpose, I will simplify the above map and segment and cluster only the neighbourhoods in **Scarborough**, So I will slice the original dataframe and create a new dataframe of the Scarborough data.

In [39]:
df_scarborough = df[df['Borough']=='Scarborough'].reset_index(drop=True)
df_scarborough.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Get the geographical coordinates of Scarborough.

In [40]:
address = 'Scarborough, Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_s = location.latitude
longitude_s = location.longitude
print(f"The geographical coordinate of Scarborough in Toronto are {latitude_s}, {longitude_s}")

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geographical coordinate of Scarborough in Toronto are 43.773077, -79.257774


As we did with all of Toronto, let's visualize Scarborough the neighbourhoods in it.

In [41]:
# create map of Toronto using latitude and longitude values.
map_scarborough = folium.Map(location=[latitude_s,longitude_s],zoom_start=10)

# add markers to map
for lat, lng, bor, nei in zip(df_scarborough['Latitude'],df_scarborough['Longitude'],df_scarborough['Borough'],df_scarborough['Neighbourhood']):
    
    label = f'{nei},{bor}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'navy',
        fill = True,
        fill_colors = '#3186cc',
        fill_opacity = 0.7,
        parse_html=False).add_to(map_scarborough)
    
map_scarborough

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

### Define Foursquare Credentials and Version

In [42]:
CLIENT_ID = 'JMEUHCFMSELYYLQVNQYWWM0B332QF0W13WIKM5XEBWZ53E23' # your Foursquare ID
CLIENT_SECRET = 'UC0SEMTM0Z52F543R2Z1P2PP1RW4JVSXDVQ41QO44YHUGTJ1' # your Foursquare Secret
VERSION = '20181018' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JMEUHCFMSELYYLQVNQYWWM0B332QF0W13WIKM5XEBWZ53E23
CLIENT_SECRET:UC0SEMTM0Z52F543R2Z1P2PP1RW4JVSXDVQ41QO44YHUGTJ1


#### Explore the first neighborhood in the dataframe.

Get the nieghbourhood's name.

In [51]:
df_scarborough.loc[15, 'Neighbourhood']

"L'Amoreaux West, Steeles West"

Get the neighbourhood's latitude and longitude.

In [52]:
neighbourhood_latitude = df_scarborough.loc[15, 'Latitude']
neighbourhood_longitude = df_scarborough.loc[15, 'Longitude']

neighbourhood_name = df_scarborough.loc[15, 'Neighbourhood']

print(f'Latidue and longitude value of {neighbourhood_name} are {neighbourhood_latitude}, {neighbourhood_longitude}')

Latidue and longitude value of L'Amoreaux West, Steeles West are 43.79952520000001, -79.3183887


#### Get the Top 100 venues that are in L'Amoreaux West, Steeles West within a radius of 500 meters.

In [53]:
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.\
            format(CLIENT_ID, 
                   CLIENT_SECRET, 
                   VERSION,
                   neighbourhood_latitude, 
                   neighbourhood_longitude, 
                   radius, 
                   LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=JMEUHCFMSELYYLQVNQYWWM0B332QF0W13WIKM5XEBWZ53E23&client_secret=UC0SEMTM0Z52F543R2Z1P2PP1RW4JVSXDVQ41QO44YHUGTJ1&v=20181018&ll=43.79952520000001,-79.3183887&radius=500&limit=100'

Sent the GET request and examine the results.

In [54]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bc85d70db04f55c420053b0'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4df28ea0e4cda09e6da0129a-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d145941735',
         'name': 'Chinese Restaurant',
         'pluralName': 'Chinese Restaurants',
         'primary': True,
         'shortName': 'Chinese'}],
       'id': '4df28ea0e4cda09e6da0129a',
       'location': {'address': '2900 Warden Ave',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Finch Ave. E',
        'distance': 72,
        'formattedAddress': ['2900 Warden Ave (at Finch Ave. E)',
         'Toronto ON M1W 2S8',
         'Canada'],
        'labe

In [55]:
results['response']['groups'][0]['items'][0]['venue']['categories'][0]['name']

'Chinese Restaurant'

All the information is in the **items** key. Define the **get_category_type** function.

In [0]:
# function that extract the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a panda dataframe.

In [60]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head(10)

,name,categories,lat,lng
0,Mr Congee Chinese Cuisine 龍粥記,Chinese Restaurant,43.798879,-79.318335
1,Phoenix Restaurant 金鳳餐廳,Chinese Restaurant,43.798198,-79.318432
2,Price Chopper,Grocery Store,43.799445,-79.318563
3,Subway,Sandwich Place,43.798983,-79.318838
4,Tim Hortons,Coffee Shop,43.798281,-79.318317
5,Shoppers Drug Mart,Pharmacy,43.799670,-79.319315
6,KFC,Fast Food Restaurant,43.798938,-79.318854
7,Yamamoto Japanese Cuisine 山本盛世,Japanese Restaurant,43.798589,-79.318558
8,McDonald's,Fast Food Restaurant,43.798880,-79.318724
9,Pizza Pizza,Pizza Place,43.797909,-79.318113


And how many venues were returned by Foursquare?

In [61]:
print(f'{nearby_venues.shape[0]} venues were returned by Foursqaure.')

15 venues were returned by Foursqaure.


## 2.Explore Neighbourhoods in Scarborough

##### Create a function to repeat the same process to all the neighbourhoods in Scarborough

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [64]:
scarborough_venues = getNearbyVenues(names = df_scarborough['Neighbourhood'],
                                    latitudes = df_scarborough['Latitude'],
                                    longitudes = df_scarborough['Longitude'])

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge


##### Let's check the size of the resulting dataframe.

In [65]:
print(scarborough_venues.shape)
scarborough_venues.head()

(94, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


##### Unqiue category

In [67]:
print(f"There are {len(scarborough_venues['Venue Category'].unique())} uniques categories.")

There are 58 uniques categories.


## 3. Analyze Each Neighbourhood

In [68]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to the dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Grocery Store,Gym,Gym Pool,Hakka Restaurant,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Movie Theater,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [69]:
scarborough_onehot.shape

(94, 59)

##### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [70]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped.head(10)

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Grocery Store,Gym,Gym Pool,Hakka Restaurant,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Movie Theater,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.25,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000,0.000,0.25,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.25,0.000000,0.000000,0.0,0.000000,0.000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.333333,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000,0.000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.333333,0.000,0.0,0.000000,0.333333,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.000
2,"Birch Cliff, Cliffside West",0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.00,0.000000,0.25,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.25,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000,0.000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.25,0.000000,0.000000,0.0,0.000000,0.000
3,Cedarbrae,0.0,0.142857,0.0,0.142857,0.142857,0.0,0.000000,0.000000,0.000000,0.00,0.142857,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.142857,0.000,0.00,0.0,0.0,0.0,0.142857,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000,0.000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.142857,0.0,0.000000,0.000
4,"Clairlea, Golden Mile, Oakridge",0.0,0.000000,0.0,0.222222,0.000000,0.0,0.000000,0.222222,0.111111,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.00,0.0,0.0,0.0,0.000000,0.111111,0.00,0.0,0.111111,0.0,0.0,0.0,0.0,0.000,0.000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.111111,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.111111,0.000000,0.0,0.000000,0.000
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.100000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.1,0.100000,0.000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.1,0.0,0.0,0.0,0.000,0.000,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.1,0.000000,0.000,0.1,0.200000,0.000000,0.100000,0.0,0.0,0.00,0.000000,0.100000,0.0,0.000000,0.000
6,"Cliffcrest, Cliffside, Scarborough Village West",0.2,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000,0.000,0.00,0.000000,0.000000,0.0,0.4,0.2,0.0,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.20,0.000000,0.000000,0.0,0.000000,0.000
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.125000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.125,0.00,0.0,0.0,0.0,0.000000,0.000000,0.25,0.0,0.000000,0.0,

In [71]:
scarborough_grouped.shape

(16, 59)

##### Print each neighborhood along with the top5 most common venues.

In [72]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'frequency']
    temp = temp.iloc[1:]
    temp['frequency'] = temp['frequency'].astype(float)
    temp = temp.round({'frequency':2})
    print(temp.sort_values('frequency', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                 venue  frequency
0       Breakfast Spot       0.25
1         Skating Rink       0.25
2               Lounge       0.25
3       Clothing Store       0.25
4  American Restaurant       0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  frequency
0           Playground       0.33
1          Coffee Shop       0.33
2                 Park       0.33
3  American Restaurant       0.00
4         Noodle House       0.00


----Birch Cliff, Cliffside West----
                   venue  frequency
0           Skating Rink       0.25
1                   Café       0.25
2  General Entertainment       0.25
3        College Stadium       0.25
4    American Restaurant       0.00


----Cedarbrae----
                 venue  frequency
0  Fried Chicken Joint       0.14
1               Bakery       0.14
2                 Bank       0.14
3   Athletics & Sports       0.14
4      Thai Restaurant       0.14


----Clairlea, Golden Mile, 

##### Put that into a pandas dataframe

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [74]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Lounge,Clothing Store,Vietnamese Restaurant,Department Store,Hakka Restaurant,Gym Pool,Gym,Grocery Store
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Coffee Shop,Playground,Park,Vietnamese Restaurant,Hakka Restaurant,Gym Pool,Gym,Grocery Store,General Entertainment,Furniture / Home Store
2,"Birch Cliff, Cliffside West",College Stadium,Skating Rink,General Entertainment,Café,Vietnamese Restaurant,Hakka Restaurant,Gym Pool,Gym,Grocery Store,Furniture / Home Store
3,Cedarbrae,Athletics & Sports,Thai Restaurant,Hakka Restaurant,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Furniture / Home Store,Electronics Store
4,"Clairlea, Golden Mile, Oakridge",Bus Line,Bakery,Intersection,Soccer Field,Ice Cream Shop,Park,Bus Station,Furniture / Home Store,Electronics Store,Fast Food Restaurant
5,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Fried Chicken Joint,Fast Food Restaurant,Chinese Restaurant,Thai Restaurant,Italian Restaurant,Noodle House,Rental Car Location,Pharmacy,Department Store
6,"Cliffcrest, Cliffside, Scarborough Village West",Motel,American Restaurant,Movie Theater,Skating Rink,Gym Pool,Gym,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Pet Store,Furniture / Home Store,Latin American Restaurant,Light Rail Station,Chinese Restaurant,Vietnamese Restaurant,Breakfast Spot,Bus Line,Hakka Restaurant
8,"East Birchmount Park, Ionview, Kennedy Park",Discount Store,Coffee Shop,Department Store,Bus Station,Chinese Restaurant,Train Station,Ice Cream Shop,Hakka Restaurant,Gym Pool,Gym
9,"Guildwood, Morningside, West Hill",Rental Car Location,Mexican Restaurant,Medical Center,Electronics Store,Breakfast Spot,Pizza Place,Fried Chicken Joint,Discount Store,Fast Food Restaurant,Vietnamese Restaurant


## 4. Cluster Neighborhoods

Run *k-means* to cluster the neighborhood into 5 clusters.

In [76]:
# set number of cluster
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood',axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluser labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 3, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [79]:
scarborough_merged = df_scarborough[0:16]

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarborough_merged.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Vietnamese Restaurant,College Stadium,Ice Cream Shop,Hakka Restaurant,Gym Pool,Gym,Grocery Store,General Entertainment,Furniture / Home Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3,Moving Target,Bar,Vietnamese Restaurant,Hakka Restaurant,Gym Pool,Gym,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Rental Car Location,Mexican Restaurant,Medical Center,Electronics Store,Breakfast Spot,Pizza Place,Fried Chicken Joint,Discount Store,Fast Food Restaurant,Vietnamese Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Insurance Office,Auto Garage,Bakery,Hakka Restaurant,Gym Pool,Gym,Athletics & Sports,Grocery Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Athletics & Sports,Thai Restaurant,Hakka Restaurant,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Furniture / Home Store,Electronics Store
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1,Jewelry Store,Playground,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Gym Pool,Gym,Grocery Store,General Entertainment,Furniture / Home Store
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,1,Discount Store,Coffee Shop,Department Store,Bus Station,Chinese Restaurant,Train Station,Ice Cream Shop,Hakka Restaurant,Gym Pool,Gym
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,1,Bus Line,Bakery,Intersection,Soccer Field,Ice Cream Shop,Park,Bus Station,Furniture / Home Store,Electronics Store,Fast Food Restaurant
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,1,Motel,American Restaurant,Movie Theater,Skating Rink,Gym Pool,Gym,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,1,College Stadium,Skating Rink,General Entertainment,Café,Vietnamese Restaurant,Hakka Restaurant,Gym Pool,Gym,Grocery Store,Furniture / Home Store


Finally, let's visulize the resulting clusters.

In [81]:
# create map
map_clusters = folium.Map(location=[latitude_s, longitude_s], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighbourhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine the Clusters

#### Cluster 1

In [82]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,0,Indian Restaurant,Pet Store,Furniture / Home Store,Latin American Restaurant,Light Rail Station,Chinese Restaurant,Vietnamese Restaurant,Breakfast Spot,Bus Line,Hakka Restaurant


#### Cluster 2

In [83]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1,Fast Food Restaurant,Vietnamese Restaurant,College Stadium,Ice Cream Shop,Hakka Restaurant,Gym Pool,Gym,Grocery Store,General Entertainment,Furniture / Home Store
2,Scarborough,1,Rental Car Location,Mexican Restaurant,Medical Center,Electronics Store,Breakfast Spot,Pizza Place,Fried Chicken Joint,Discount Store,Fast Food Restaurant,Vietnamese Restaurant
3,Scarborough,1,Coffee Shop,Korean Restaurant,Insurance Office,Auto Garage,Bakery,Hakka Restaurant,Gym Pool,Gym,Athletics & Sports,Grocery Store
4,Scarborough,1,Athletics & Sports,Thai Restaurant,Hakka Restaurant,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Furniture / Home Store,Electronics Store
5,Scarborough,1,Jewelry Store,Playground,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Gym Pool,Gym,Grocery Store,General Entertainment,Furniture / Home Store
6,Scarborough,1,Discount Store,Coffee Shop,Department Store,Bus Station,Chinese Restaurant,Train Station,Ice Cream Shop,Hakka Restaurant,Gym Pool,Gym
7,Scarborough,1,Bus Line,Bakery,Intersection,Soccer Field,Ice Cream Shop,Park,Bus Station,Furniture / Home Store,Electronics Store,Fast Food Restaurant
8,Scarborough,1,Motel,American Restaurant,Movie Theater,Skating Rink,Gym Pool,Gym,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint
9,Scarborough,1,College Stadium,Skating Rink,General Entertainment,Café,Vietnamese Restaurant,Hakka Restaurant,Gym Pool,Gym,Grocery Store,Furniture / Home Store
11,Scarborough,1,Middle Eastern Restaurant,Shopping Mall,Breakfast Spot,Sandwich Place,Vietnamese Restaurant,Auto Garage,Grocery Store,Gym,College Stadium,Gym Pool


#### Cluster 3

In [84]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Scarborough,2,Pizza Place,Fried Chicken Joint,Fast Food Restaurant,Chinese Restaurant,Thai Restaurant,Italian Restaurant,Noodle House,Rental Car Location,Pharmacy,Department Store


#### Cluster 4

In [85]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3,Moving Target,Bar,Vietnamese Restaurant,Hakka Restaurant,Gym Pool,Gym,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint
15,Scarborough,3,Coffee Shop,Chinese Restaurant,Fast Food Restaurant,Sandwich Place,Pharmacy,Pizza Place,Gym Pool,Japanese Restaurant,Grocery Store,Gym


#### Cluster 5

In [86]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,4,Coffee Shop,Playground,Park,Vietnamese Restaurant,Hakka Restaurant,Gym Pool,Gym,Grocery Store,General Entertainment,Furniture / Home Store
